In [1]:
from utils.structure import *
from utils.predicter import *
from utils.solver import *

In [3]:
# build structure and display
test_datas = open('./data/test_demo.txt', 'r')
test_datas = test_datas.readlines()
for test_data in test_datas:
    s = Structure(test_data)
#     print(s.data)
#     print(s.display())

In [15]:
s = Structure(test_datas[7])
print(s.display())

solver = BasicSolver(s)
# solver.idxes_need_to_solve

+-----+-----+-----+
|. . 6|4 . 9|. . 5|
|     |     |     |
|1 8 4|2 . .|9 6 3|
|     |     |     |
|5 . .|3 6 .|4 . .|
+-----+-----+-----+
|. . 3|. . 4|. . 9|
|     |     |     |
|7 . 8|. . .|3 4 2|
|     |     |     |
|4 . .|8 . .|5 . .|
+-----+-----+-----+
|. . .|9 4 6|. 5 1|
|     |     |     |
|9 . 5|. . 2|. 3 4|
|     |     |     |
|. 4 .|5 . .|2 9 .|
+-----+-----+-----+



In [16]:
# solver.check_idx_only(62)
# print(solver.ready)
# solver.check_idx_last_left(62)
# print(solver.ready)
print(solver.check_scanned_drop('5'))
print(solver.check_area_drop('5'))
print(solver.ready)
solver.update()
print(solver.ready)

+-----+-----+-----+
|    6|4   9|    5|
|     |     |     |
|1 8 4|2 . .|9 6 3|
|     |     |     |
|5    |3 6  |4    |
+-----+-----+-----+
|  . 3|  . 4|    9|
|     |     |     |
|7 . 8|  . .|3 4 2|
|     |     |     |
|4    |8    |5    |
+-----+-----+-----+
|     |9 4 6|  5 1|
|     |     |     |
|9   5|    2|  3 4|
|     |     |     |
|  4  |5    |2 9  |
+-----+-----+-----+

False
+-----+-----+-----+
|    6|4   9|    5|
|     |     |     |
|1 8 4|2    |9 6 3|
|     |     |     |
|5    |3 6  |4    |
+-----+-----+-----+
|    3|  . 4|    9|
|     |     |     |
|7   8|  . .|3 4 2|
|     |     |     |
|4    |8    |5    |
+-----+-----+-----+
|     |9 4 6|  5 1|
|     |     |     |
|9   5|    2|  3 4|
|     |     |     |
|  4  |5    |2 9  |
+-----+-----+-----+

True
[(31, '5'), (41, '5')]
[]


In [13]:
print(solver.structure.display(solver.data))

+-----+-----+-----+
|. . 6|4 . 9|. . 5|
|     |     |     |
|1 8 4|2 . .|9 6 3|
|     |     |     |
|5 . .|3 6 .|4 . .|
+-----+-----+-----+
|. . 3|. 5 4|. . 9|
|     |     |     |
|7 . 8|. . 5|3 4 2|
|     |     |     |
|4 . .|8 . .|5 . .|
+-----+-----+-----+
|. . .|9 4 6|. 5 1|
|     |     |     |
|9 . 5|. . 2|. 3 4|
|     |     |     |
|. 4 .|5 . .|2 9 .|
+-----+-----+-----+



In [ ]:
if not len(solver.ready) > 0:
    print('s')
else:
    print('t')

In [ ]:
solver.steps.extend(solver.ready)

In [ ]:
solver.steps

In [ ]:
tmp = " 	 	6	4	 	9	 	 	5\
	1	8	4	2	 	 	9	6	3\
	5	 	 	3	6	 	4	 	 \
	 	 	3	 	 	4	 	 	9\
	7	 	8	 	 	 	3	4	2\
	4	 	 	8	 	 	5	 	 \
	?	×	×	9	4	6	 	5	1\
	9	×	5	 	 	2	 	3	4\
	?	4	×	5	 	 	2	9	 "

In [4]:
tmp = tmp.replace(' ', '.').replace('×', '.').replace('?','').replace('	', '')
print(len(','.join(list(tmp))))
','.join(list(tmp))

NameError: name 'tmp' is not defined